In [26]:
# Setup
if __name__ == '__main__':
    import os
    # Change the current working directory to the parent directory of this file
    os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(__vsc_ipynb_file__))))

from evaluation import get_actual_demand, get_known
from system_state import SystemState
from utils import load_problem_data, load_json
import numpy as np
import pandas as pd
import uuid
import json

demand, datacenters, servers, selling_prices = load_problem_data()
datacenters = datacenters.set_index('datacenter_id')
servers = servers.set_index('server_generation')

seed = 4507
np.random.seed(seed)

moveless_actions = pd.DataFrame(load_json('./greedy_profit_v2/proof_of_concept_notebooks/pre_move_actions_4507.json'))
moveless_results = pd.DataFrame(load_json('./greedy_profit_v2/proof_of_concept_notebooks/pre_move_results_4507.json'))

def dataframe_lookup(dataframe: pd.DataFrame, key_column: str, key_value: any, value_column: str):
    return dataframe.query(f"{key_column} == @key_value")[value_column].iloc[0]

In [36]:
actions = moveless_actions.copy()
results = moveless_results.copy()
results['buy_time_step'] = results['buy_time_step'].astype(int)
results['dismiss_time_step'] = results['dismiss_time_step'].astype(int)

BASE_MOVE_THRESHOLD = 10 # time steps

# 1) For each server generation's results
# for server_generation in get_known('server_generation'):
server_generation = 'GPU.S1'
server_gen_results = results.query("server_generation == @server_generation")
print(server_gen_results)


# 1) Sort results by operating time (dismiss time step - buy time step)
sorted_server_gen_results = server_gen_results.eval('operating_time = dismiss_time_step - buy_time_step').sort_values(by='operating_time')
# print(sorted_server_gen_results)
# 2) Filter results with an operating time > life expectancy
sorted_server_gen_results: pd.DataFrame = sorted_server_gen_results.query('operating_time < 96')
print(sorted_server_gen_results)


# 1) For each result (from shortest to longest server operating time)
for i, result in sorted_server_gen_results.iterrows():

    # Jamie's MOVE_THRESHOLD is based on the number of time steps before you moving doesn't generate any profit
    energy_cost_per_time_step = datacenters.loc[result['datacenter_id']]['cost_of_energy'] * servers.loc[server_generation]['energy_consumption']
    average_maintenance = servers.loc[server_generation]['average_maintenance_fee']
    cost_of_moving = 1000
    purchase_price = servers.loc[server_generation]['purchase_price']
    MOVE_THRESHOLD = int(np.round((purchase_price - cost_of_moving) / (average_maintenance + energy_cost_per_time_step)))

    # 1) Search for a result of the same server generation different datacenter which has a dismiss time step briefly before or after this result's buy time step (based on some threshold e.g. >= 5 time steps before)
    minimum_dismiss_time_step = result['buy_time_step'] - MOVE_THRESHOLD
    maximum_dismiss_time_step = result['buy_time_step'] + BASE_MOVE_THRESHOLD
    candidate_move_to = sorted_server_gen_results.query('dismiss_time_step >= @minimum_dismiss_time_step and dismiss_time_step <= @maximum_dismiss_time_step and dismiss_time_step < @result["dismiss_time_step"] and datacenter_id != @result["datacenter_id"]')

    if candidate_move_to.empty:
        print(f"Result {result['buy_time_step']}:{result['dismiss_time_step']} (MOVE_THRESHOLD={MOVE_THRESHOLD}) has no candidate merges")
        continue

    
    # 2) Validate that the difference between the earlier buy time step and the later dismiss time step doesn't exceed the life expectancy (96)
    # TODO

    
    # 3) Calculate the number of servers to move: the minimum buy count of either result
    # TODO
    print(f"Result {result['buy_time_step']}:{result['dismiss_time_step']} (MOVE_THRESHOLD={MOVE_THRESHOLD}) could merge with {candidate_move_to}")

    
    # 4) Find the server IDs of the servers to move from the actions array
    # TODO

    
    # 5) Construct move actions for each server to move to replace the buy/dismiss actions.
    # TODO

    
        # 1) If the datacenter moved from has better energy efficiency, perform the move at the same time step as the dismiss
        # TODO

    
        # 2) If the datacenter moved to has better energy efficiency, perform the move at the same time step as the buy
        # TODO
    

    # 6) Delete the corresponding buy and dismiss actions from the actions array
    # TODO

   server_generation buy_count datacenter_id  buy_time_step  dismiss_time_step
10            GPU.S1       159           DC3              1                109
11            GPU.S1       427           DC3             18                 81
12            GPU.S1        83           DC3             39                 81
13            GPU.S1        70           DC3             43                 73
17            GPU.S1        35           DC2              1                109
18            GPU.S1        30           DC2             37                106
23            GPU.S1        63           DC1              1                109
24            GPU.S1        41           DC1             30                105
25            GPU.S1        32           DC1             64                 93
   server_generation buy_count datacenter_id  buy_time_step  \
25            GPU.S1        32           DC1             64   
13            GPU.S1        70           DC3             43   
12            GPU.S1 